In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cartopy.feature import ShapelyFeature
import cartopy.crs as ccrs
import matplotlib.patches as mpatches

# ---------------------------------------------------------------------------------------------------------------------
# Load the counties and wards data
counties = gpd.read_file('data_files/Counties.shp')
wards = gpd.read_file('data_files/Wards.shp')

# Ensure both datasets are in the same projection (UTM projection for consistency)
wards = wards.to_crs(epsg=32629)
counties = counties.to_crs(epsg=32629)

# Perform spatial join to associate wards with counties
wards_county = gpd.sjoin(wards, counties, how='inner', predicate='intersects')

# Summarize total population by county
population_by_county = wards_county.groupby('CountyName')['Population'].sum().reset_index()

# Print county with highest and lowest population
max_county = population_by_county.loc[population_by_county['Population'].idxmax()]
min_county = population_by_county.loc[population_by_county['Population'].idxmin()]

print("County with highest population: {} - {} residents".format(max_county['CountyName'], max_county['Population']))
print("County with lowest population: {} - {} residents".format(min_county['CountyName'], min_county['Population']))

# ---------------------------------------------------------------------------------------------------------------------
# Create the map
ni_utm = ccrs.UTM(29)

fig, ax = plt.subplots(1, 1, figsize=(10, 10), subplot_kw=dict(projection=ni_utm))

# Add gridlines
gridlines = ax.gridlines(draw_labels=True, xlocs=[-8, -7.5, -7, -6.5, -6, -5.5], ylocs=[54, 54.5, 55, 55.5])
gridlines.right_labels = False
gridlines.bottom_labels = False

# Add colorbar for population
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1, axes_class=plt.Axes)

# Plot the ward population data
ward_plot = wards.plot(column='Population', ax=ax, vmin=1000, vmax=8000, cmap='viridis',
                       legend=True, cax=cax, legend_kwds={'label': 'Resident Population'})

# Add county boundaries in red
county_outlines = ShapelyFeature(counties['geometry'], ni_utm, edgecolor='r', facecolor='none')
ax.add_feature(county_outlines)

# Function to generate legend handles
def generate_handles(labels, colors, edge='k'):
    return [mpatches.Patch(facecolor=color, edgecolor=edge, label=label) for label, color in zip(labels, colors)]

county_handles = generate_handles([''], ['none'], edge='r')

# Add a legend for county boundaries
ax.legend(county_handles, ['County Boundaries'], fontsize=12, loc='upper left', framealpha=1)

# Save the map
fig.savefig('population_map.png', dpi=300, bbox_inches='tight')

print("Map saved as population_map.png")
